In [9]:
import os, sys
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, os.path.abspath('..'))

from dotenv import load_dotenv
load_dotenv('../.env')

print('✅ Ready' if os.getenv('GROQ_API_KEY') else '❌ GROQ_API_KEY not found')

✅ Ready


In [10]:
from langchain_core.messages import HumanMessage
from src.state import AgentState
from src.graph import create_graph
from src.tools.financial_tools import calculate_rsi, calculate_macd, calculate_sma

print('✅ Modules loaded')

✅ Modules loaded


## Test Tools

In [11]:
import yfinance as yf

TICKER = "AAPL"
df = yf.Ticker(TICKER).history(period="3mo")

df['SMA_20'] = calculate_sma(df)
df['RSI_14'] = calculate_rsi(df)

last = df.iloc[-1]
print(f"Price: ${last['Close']:.2f} | SMA20: ${last['SMA_20']:.2f} | RSI: {last['RSI_14']:.1f}")

Price: $278.78 | SMA20: $274.76 | RSI: 58.9


## Run Analysis

In [12]:
def analyze(ticker: str) -> dict:
    print(f"\n🚀 Analyzing {ticker}...")
    graph = create_graph()
    
    state: AgentState = {
        "messages": [HumanMessage(content=f"Analyze stock {ticker}")],
        "next": "Supervisor",
        "current_ticker": ticker,
        "current_stock_data": {},
        "iteration_count": 0,
        "verification_passed": False,  # PEV Pattern
        "verification_result": {},     # PEV Pattern
    }
    
    result = None
    for event in graph.stream(state, config={"recursion_limit": 25}):
        for node, output in event.items():
            print(f"  ✓ {node}")
            result = output
    
    print("✅ Done")
    return result

In [13]:
result = analyze("AAPL")


🚀 Analyzing AAPL...
  ✓ Supervisor
  ✓ Supervisor


/Users/hlaff/ai-engineer/hedge_fund_bot/src/tools/search_tools.py:15: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)
/Users/hlaff/ai-engineer/hedge_fund_bot/src/tools/search_tools.py:15: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)
/Users/hlaff/ai-engineer/hedge_fund_bot/src/tools/search_tools.py:15: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)


  ✓ Researcher
  ✓ Supervisor
  ✓ Supervisor
  ✓ Chartist
  ✓ Chartist
  ✓ Supervisor
  ✓ Supervisor
  ✓ Analyst
  ✓ Analyst
  ✓ Supervisor
✅ Done
  ✓ Supervisor
✅ Done


In [14]:
# Display report
if result and "messages" in result:
    for msg in result["messages"]:
        if "[ANALYST" in msg.content:
            display(Markdown(msg.content.replace("[ANALYST - FINAL REPORT]\n\n", "")))
            break

In [15]:
print(result)

{'next': 'FINISH', 'messages': [HumanMessage(content='[SUPERVISOR] Next: FINISH', additional_kwargs={}, response_metadata={})]}
